# Library

In [ ]:
import yfinance as yf
import pandas as pd

import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Input
from sklearn.preprocessing import MinMaxScaler

## Train

In [ ]:
def read_yfinance():
    nvda = yf.Ticker("NVDA").history(period="1mo")#.reset_index()
    # nvda['Date'] = pd.to_datetime(nvda['Date']).dt.date
    nvda = nvda.drop(["Dividends", "Stock Splits"], axis=1)
    return nvda

data = pd.DataFrame(read_yfinance()['Close'])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data)-time_step-1):
        X.append(data[i:(i+time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 3
X, y = create_dataset(scaled_data, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)

In [10]:
model = Sequential()
model.add(Input(shape=(X.shape[1], 1)))
model.add(LSTM(units=50, activation='relu', return_sequences=False))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 50)             │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
model.fit(X, y, epochs=20, batch_size=15)

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.26049
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2818  
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2581
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2493  
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2480 
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2328
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1950 
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2014  
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1988  
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1845  
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1859
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1540
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1544
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1446   
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1390 
Epoch 16/20
2/2 ━━━━━━━━━━━━━━━━

In [5]:
model.save('teste_model.keras')

## Predict

In [ ]:

#load model
load_teste_model = load_model('teste_model.keras')

def read_yfinance():
    nvda = yf.Ticker("NVDA").history(period="5d")#.reset_index()
    # nvda['Date'] = pd.to_datetime(nvda['Date']).dt.date
    nvda = nvda.drop(["Dividends", "Stock Splits"], axis=1)
    return nvda

data = pd.DataFrame(read_yfinance()['Close'])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

time_step = 3
X = []

for i in range(len(scaled_data)-time_step):
    X.append(scaled_data[i:(i+time_step), 0])

X = np.array(X)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [19]:
prediction = load_teste_model.predict(X)
prediction = scaler.inverse_transform(prediction)
print(prediction[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[139.16919]
